<center>Myranda Swartzwelter<br>
    DSC-550 T301 Data Mining<br>
    Week 3 Assignment: NLP<br>
 3/29/22
   

1. Import the movie review data as a data frame and ensure that the data is loaded properly.

In [3]:
#import necessary packages
import pandas as pd
import numpy as np
from textblob import TextBlob 

In [8]:
#import file note it is a tsv
reviews_train = pd.read_csv('labeledTrainData.tsv', sep='\t')

#confirm that data is properly loaded - should have 25000 rows and looking at the first few rows should show ID, sentiment and review
reviews_train.head()
reviews_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25000 entries, 0 to 24999
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   id         25000 non-null  object
 1   sentiment  25000 non-null  int64 
 2   review     25000 non-null  object
dtypes: int64(1), object(2)
memory usage: 586.1+ KB


2. How many of each positive and negative reviews are there?

In [14]:
#count the number of rows grouping by sentiment
reviews_train.groupby(by='sentiment')['id'].count()

#there are an equal number of positive and negative reviews - 12500 of each

sentiment
0    12500
1    12500
Name: id, dtype: int64

3. Use TextBlob to classify each movie review as positive or negative. Assume that a polarity score greater than or equal to zero is a positive sentiment and less than 0 is a negative sentiment.



In [25]:
#create an array to list the sentiments we generate
sentiments = []

#iterate through dataframe and get the polarity for each review
for index, row in reviews_train.iterrows():
    blob = TextBlob(row['review'])
    polarity = blob.sentiment.polarity
    
    #if the polarity score >= 0 this is a positive sentiment and <0 is a negative sentiment
    if polarity < 0:
        sentiment = 0
    else:
        sentiment = 1
    #add sentiment to list
    sentiments.append(sentiment)

#create a new colum with the modeled sentiments
reviews_train['modeled_sentiments'] = sentiments
reviews_train.head()

,id,sentiment,review,modeled_sentiments
0,5814_8,1,With all this stuff going down at the moment w...,1
1,2381_9,1,"\The Classic War of the Worlds\"" by Timothy Hi...",1
2,7759_3,0,The film starts with a manager (Nicholas Bell)...,0
3,3630_4,0,It must be assumed that those who praised this...,1
4,9495_8,1,Superbly trashy and wondrously unpretentious 8...,0


4. Check the accuracy of this model. Is this model better than random guessing?

In [29]:
#create a new column for values that are correct
reviews_train['correct'] = np.where(reviews_train['sentiment']== reviews_train['modeled_sentiments'], 1, 0)

#count number of correct by model
total_correct = reviews_train['correct'].sum()

#number of reviews
total = reviews_train['correct'].count()

#calculate accuracy
accuracy = total_correct / total *100
print(total_correct, total, accuracy)

17131 25000 68.524


5. For up to five points extra credit, use another prebuilt text sentiment analyzer, e.g., VADER, and repeat steps (3) and (4).

In [37]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

#create an array to list the sentiments we generate
sentiments_vader = []
sid_obj = SentimentIntensityAnalyzer()

#iterate through dataframe and get the polarity for each review
for index, row in reviews_train.iterrows():
    review = row['review']
    compound = sid_obj.polarity_scores(review)['compound']
    
    #if the polarity score >= 0 this is a positive sentiment and <0 is a negative sentiment
    #Not accounting for neutral (compound = 0)
    if compound < 0:
        sentiment = 0
    else:
        sentiment = 1
    #add sentiment to list
    sentiments_vader.append(sentiment)

#create a new colum with the modeled sentiments
reviews_train['vader_sentiments'] = sentiments_vader
reviews_train.head()

,id,sentiment,review,modeled_sentiments,correct,vader_sentiments
0,5814_8,1,With all this stuff going down at the moment w...,1,1,0
1,2381_9,1,"\The Classic War of the Worlds\"" by Timothy Hi...",1,1,1
2,7759_3,0,The film starts with a manager (Nicholas Bell)...,0,1,0
3,3630_4,0,It must be assumed that those who praised this...,1,0,0
4,9495_8,1,Superbly trashy and wondrously unpretentious 8...,0,0,1


In [39]:
#create a new column for values that are correct
reviews_train['correct_vader'] = np.where(reviews_train['sentiment']== reviews_train['vader_sentiments'], 1, 0)

#count number of correct by model
total_correct = reviews_train['correct_vader'].sum()

#number of reviews
total = reviews_train['correct_vader'].count()

#calculate accuracy
accuracy = total_correct / total *100
print(total_correct, total, accuracy)

17351 25000 69.404
